In [1]:
!pip install langchain
!pip install langchain_core
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.0 MB/s eta 0:00:00


In [4]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool, StructuredTool
from langchain_openai import ChatOpenAI

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("openai_api_key")

In [5]:

def greet_user(name: str) -> str:
  return f"Hello, {name}!"

def reverse_string(text: str) -> str:
  return text[::-1]

def concatenate_string(a: str, b: str) -> str:
  return a + b

class ConcatenateStringArgs(BaseModel):
  a: str = Field(description="First String")
  b: str = Field(description="Second String")

tools = [
    Tool(
        name="GreetUser",
        func=greet_user,
        description="Greet the user by name."
    ),
    Tool(
        name="ReverseString",
        func=reverse_string,
        description="Reverse the given string."
    ),
    StructuredTool.from_function(
        func=concatenate_string,
        name="ConcatenateStrings",
        description="Concatenate two strings.",
        args_schema=ConcatenateStringArgs,
    ),
]

llm = ChatOpenAI(model="gpt-4o")

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

response = agent_executor.invoke({"input": "Greet Ravi"})
print("Response for 'Greet Ravi':", response)

response = agent_executor.invoke({"input": "Reverse the string 'hello'"})
print("Response for 'Reverse the string hello':", response)

response = agent_executor.invoke({"input": "Concatenate 'hello' and 'world'"})
print("Response for 'Concatenation':", response)



/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `GreetUser` with `Ravi`


Hello, Ravi!Hello, Ravi!

> Finished chain.
Response for 'Greet Ravi': {'input': 'Greet Ravi', 'output': 'Hello, Ravi!'}


> Entering new AgentExecutor chain...

Invoking: `ReverseString` with `hello`


ollehThe reverse of the string "hello" is "olleh".

> Finished chain.
Response for 'Reverse the string hello': {'input': "Reverse the string 'hello'", 'output': 'The reverse of the string "hello" is "olleh".'}


> Entering new AgentExecutor chain...

Invoking: `ConcatenateStrings` with `{'a': 'hello', 'b': 'world'}`


helloworldThe concatenated result of "hello" and "world" is "helloworld".

> Finished chain.
Response for 'Concatenation': {'input': "Concatenate 'hello' and 'world'", 'output': 'The concatenated result of "hello" and "world" is "helloworld".'}


In [7]:
#decorator
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field

@tool()
def greet_user(name: str) -> str:
    """Greet the user by name."""
    return f"Hello, {name}!"

class ReverseStringArgs(BaseModel):
    text: str = Field(description="Text to be reversed")

@tool(args_schema=ReverseStringArgs)
def reverse_string(text: str) -> str:
    """Reverse the given string."""
    return text[::-1]

class ConcatenateStringsArgs(BaseModel):
    a: str = Field(description="First string")
    b: str = Field(description="Second string")

@tool(args_schema=ConcatenateStringsArgs)
def concatenate_strings(a: str, b: str) -> str:
    """Concatenate two strings."""
    print("a", a)
    print("b", b)
    return a + b

tools = [
    greet_user,
    reverse_string,
    concatenate_strings,
]


response = agent_executor.invoke({"input": "Greet Alice"})
print("Response for 'Greet Alice':", response)

response = agent_executor.invoke({"input": "Reverse the string 'hello'"})
print("Response for 'Reverse the string hello':", response)

response = agent_executor.invoke({"input": "Concatenate 'hello' and 'world'"})
print("Response for 'Concatenate hello and world':", response)



> Entering new AgentExecutor chain...

Invoking: `GreetUser` with `Alice`


Hello, Alice!Hello, Alice!

> Finished chain.
Response for 'Greet Alice': {'input': 'Greet Alice', 'output': 'Hello, Alice!'}


> Entering new AgentExecutor chain...

Invoking: `ReverseString` with `hello`


ollehThe reversed string of 'hello' is 'olleh'.

> Finished chain.
Response for 'Reverse the string hello': {'input': "Reverse the string 'hello'", 'output': "The reversed string of 'hello' is 'olleh'."}


> Entering new AgentExecutor chain...

Invoking: `ConcatenateStrings` with `{'a': 'hello', 'b': 'world'}`


helloworldThe concatenation of 'hello' and 'world' is 'helloworld'.

> Finished chain.
Response for 'Concatenate hello and world': {'input': "Concatenate 'hello' and 'world'", 'output': "The concatenation of 'hello' and 'world' is 'helloworld'."}
